In [1]:
import os

print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 6\IEEE Access\DroMoLog\notebooks


In [2]:
import os
os.chdir("..")

In [3]:
print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 6\IEEE Access\DroMoLog


In [4]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from utils.preprocessing import BaselineDataset
from transformers import BertModel, BertTokenizer

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
idx2label = {
        1: 'Normal',
        2: 'Low',
        3: 'Medium',
        4: 'High'
    }

dataset_path = 'dataset/merged-manual-unique.csv'
label_encoder_multi = LabelEncoder()
dataset = pd.read_csv(dataset_path)
dataset["label"] = dataset['label'].map(idx2label)
dataset["multiclass_label"] = label_encoder_multi.fit_transform(dataset["label"].to_list())

bert_model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name).to(device)

max_seq_length = 65
batch_size = 8

merged_dataset = BaselineDataset(dataset, tokenizer, max_seq_length)
merged_loader = DataLoader(merged_dataset, batch_size=batch_size, shuffle=False)

all_labels_multiclass = []
all_embeddings = []

with torch.no_grad():
        for batch in merged_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels_index = batch["labels_index"]
    
            embeddings = bert_model(input_ids, attention_mask).last_hidden_state[:, 0, :]
            all_labels_multiclass.extend(labels_index)
            all_embeddings.append(embeddings)

tsne = TSNE(n_components=2, random_state=42)
all_embeddings = torch.cat(all_embeddings, dim=0)
reduced_embeddings = tsne.fit_transform(all_embeddings.cpu().numpy())
label_decoded = label_encoder_multi.inverse_transform(all_labels_multiclass)
labels = list(label_decoded)

plt.figure(figsize=(5, 2.5))
fig, ax = plt.subplots()

unique_labels = ['Normal', 'Low', 'Medium', 'High']
colors = ['#4CAF50', '#FFC107', '#FF5722', '#D32F2F']

counter = 0
for label in unique_labels:
    # Filter data points for each unique label
    x_filtered = [reduced_embeddings[i][0] for i in range(len(reduced_embeddings)) if labels[i] == label]
    y_filtered = [reduced_embeddings[i][1] for i in range(len(reduced_embeddings)) if labels[i] == label]
    ax.scatter(x_filtered, y_filtered, label=label, s=15, c=colors[counter])
    counter+=1

# Add a legend with only unique labels
ax.set_xticks([])
ax.set_yticks([])
legend = ax.legend(loc='upper right')

# Display the plot
plt.savefig(os.path.join('visualization', "dataset_viz.pdf"), bbox_inches='tight')
plt.show()
plt.close()

c:\Users\Swardi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[]

<Figure size 500x250 with 0 Axes>